## Імпортуємо бібліотеки

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD

In [ ]:
#Convert date
def to_datetime(df):
    date = datetime.strptime(df, '%d.%m.%Y')
    return date.strftime("%Y-%m-%d")

## Завантажуємо дані

In [ ]:
df = pd.read_csv('../input/tesla-stock-price-new/Price Tesla.csv', sep=';')
df['Date'] = df['Date'].apply(lambda x: to_datetime(x))
df = df.sort_values('Date').reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.shape

> ### Виконуємо prediction за стовпчиком "Price"

In [ ]:
df['Price'] = df['Price'].astype(float)

plt.figure(figsize=(20,7))
plt.plot(df['Date'].values, df['Price'].values, label = 'Tesla Stock Price', color = 'red')
plt.xticks(np.arange(100,df.shape[0],200))
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

## Розбиваємо на навчальну та тестову частини

In [ ]:
num_shape = 482

train = df.iloc[:num_shape, 1:2].values
test = df.iloc[num_shape:, 1:2].values

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
train_scaled = sc.fit_transform(train)

In [ ]:
X_train = []

#Price on next day
y_train = []

window = 60

for i in range(window, num_shape):
    X_train_ = np.reshape(train_scaled[i-window:i, 0], (window, 1))
    X_train.append(X_train_)
    y_train.append(train_scaled[i, 0])
X_train = np.stack(X_train)
y_train = np.stack(y_train)

## Перша модель - з шаром базових рекурентних нейронів

In [ ]:
model = Sequential()

model.add(SimpleRNN(units = 50,  input_shape = (X_train.shape[1], 1)))

model.add(Dense(units = 1))
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(X_train, y_train, epochs = 200, batch_size = 32);

In [ ]:
df_volume = np.vstack((train, test))

inputs = df_volume[df_volume.shape[0] - test.shape[0] - window:]
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

num_2 = df_volume.shape[0] - num_shape + window

X_test = []

for i in range(window, num_2):
    X_test_ = np.reshape(inputs[i-window:i, 0], (window, 1))
    X_test.append(X_test_)
    
X_test = np.stack(X_test)

In [ ]:
predict = model.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test

print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))
print("MAPE:", np.mean(np.abs((test - predict) / test)) * 100)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[450:], df_volume[450:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df['Date'][-predict.shape[0]:].values, predict, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[450:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

> ### Передбачення на 20 днів

In [ ]:
pred_ = predict[-1].copy()
prediction_full = []
window = 60
df_copy = df.iloc[:, 1:2][1:].values

for j in range(20):
    df_ = np.vstack((df_copy, pred_))
    train_ = df_[:num_shape]
    test_ = df_[num_shape:]
    
    df_volume_ = np.vstack((train_, test_))

    inputs_ = df_volume_[df_volume_.shape[0] - test_.shape[0] - window:]
    inputs_ = inputs_.reshape(-1,1)
    inputs_ = sc.transform(inputs_)

    X_test_2 = []

    for k in range(window, num_2):
        X_test_3 = np.reshape(inputs_[k-window:k, 0], (window, 1))
        X_test_2.append(X_test_3)

    X_test_ = np.stack(X_test_2)
    predict_ = model.predict(X_test_)
    pred_ = sc.inverse_transform(predict_)
    prediction_full.append(pred_[-1][0])
    df_copy = df_[j:]

In [ ]:
prediction_full_new = np.vstack((predict, np.array(prediction_full).reshape(-1,1)))

In [ ]:
df_date = df[['Date']]

for h in range(20):
    df_date_add = pd.to_datetime(df_date['Date'].iloc[-1]) + pd.DateOffset(days=1)
    df_date_add = pd.DataFrame([df_date_add.strftime("%Y-%m-%d")], columns=['Date'])
    df_date = df_date.append(df_date_add)
df_date = df_date.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[300:], df_volume[300:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df_date['Date'][-prediction_full_new.shape[0]:].values, prediction_full_new, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[300:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

---------------------------------

## Проста модель з шаром LSTM

In [ ]:
modelLSTM = Sequential()

modelLSTM.add(LSTM(units = 50,  input_shape = (X_train.shape[1], 1)))

modelLSTM.add(Dense(units = 1))
modelLSTM.summary()

In [ ]:
modelLSTM.compile(optimizer = 'adam', loss = 'mean_squared_error')
modelLSTM.fit(X_train, y_train, epochs = 200, batch_size = 32);

In [ ]:
predict = modelLSTM.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test

print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))
print("MAPE:", np.mean(np.abs((test - predict) / test)) * 100)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[450:], df_volume[450:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df['Date'][-predict.shape[0]:].values, predict, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[450:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

> ### Передбачення на 20 днів

In [ ]:
pred_ = predict[-1].copy()
prediction_full = []
window = 60
df_copy = df.iloc[:, 1:2][1:].values

for j in range(20):
    df_ = np.vstack((df_copy, pred_))
    train_ = df_[:num_shape]
    test_ = df_[num_shape:]
    
    df_volume_ = np.vstack((train_, test_))

    inputs_ = df_volume_[df_volume_.shape[0] - test_.shape[0] - window:]
    inputs_ = inputs_.reshape(-1,1)
    inputs_ = sc.transform(inputs_)

    X_test_2 = []
    
    for k in range(window, num_2):
        X_test_3 = np.reshape(inputs_[k-window:k, 0], (window, 1))
        X_test_2.append(X_test_3)

    X_test_ = np.stack(X_test_2)
    predict_ = model.predict(X_test_)
    pred_ = sc.inverse_transform(predict_)
    prediction_full.append(pred_[-1][0])
    df_copy = df_[j:]

In [ ]:
prediction_full_new = np.vstack((predict, np.array(prediction_full).reshape(-1,1)))

df_date = df[['Date']]

for h in range(20):
    df_date_add = pd.to_datetime(df_date['Date'].iloc[-1]) + pd.DateOffset(days=1)
    df_date_add = pd.DataFrame([df_date_add.strftime("%Y-%m-%d")], columns=['Date'])
    df_date = df_date.append(df_date_add)
df_date = df_date.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[300:], df_volume[300:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df_date['Date'][-prediction_full_new.shape[0]:].values, prediction_full_new, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[300:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

----------------------

## Проста модель з шаром GRU

In [ ]:
modelGRU = Sequential()

modelGRU.add(GRU(units=50, input_shape=(X_train.shape[1],1)))

modelGRU.add(Dense(units=1))
modelGRU.summary()

In [ ]:
modelGRU.compile(optimizer='sgd', loss='mean_squared_error')
modelGRU.fit(X_train, y_train, epochs=200, batch_size=16)

In [ ]:
predict = modelGRU.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test


print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))
print("MAPE:", np.mean(np.abs((test - predict) / test)) * 100)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[450:], df_volume[450:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df['Date'][-predict.shape[0]:].values, predict, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[450:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

> ### Передбачення на 20 днів

In [ ]:
pred_ = predict[-1].copy()
prediction_full = []
window = 60
df_copy = df.iloc[:, 1:2][1:].values

for j in range(20):
    df_ = np.vstack((df_copy, pred_))
    train_ = df_[:num_shape]
    test_ = df_[num_shape:]
    
    df_volume_ = np.vstack((train_, test_))

    inputs_ = df_volume_[df_volume_.shape[0] - test_.shape[0] - window:]
    inputs_ = inputs_.reshape(-1,1)
    inputs_ = sc.transform(inputs_)

    X_test_2 = []

    for k in range(window, num_2):
        X_test_3 = np.reshape(inputs_[k-window:k, 0], (window, 1))
        X_test_2.append(X_test_3)

    X_test_ = np.stack(X_test_2)
    predict_ = modelGRU.predict(X_test_)
    pred_ = sc.inverse_transform(predict_)
    prediction_full.append(pred_[-1][0])
    df_copy = df_[j:]

In [ ]:
prediction_full_new = np.vstack((predict, np.array(prediction_full).reshape(-1,1)))

df_date = df[['Date']]

for h in range(20):
    kk = pd.to_datetime(df_date['Date'].iloc[-1]) + pd.DateOffset(days=1)
    kk = pd.DataFrame([kk.strftime("%Y-%m-%d")], columns=['Date'])
    df_date = df_date.append(kk)
df_date = df_date.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[300:], df_volume[300:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df_date['Date'][-prediction_full_new.shape[0]:].values, prediction_full_new, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df_date[300:].shape[0],20))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

-----------------------

## Модель LSTM з шаром TimeDistributed

In [ ]:
from keras.layers import TimeDistributed

In [ ]:
modeLSTM_time = Sequential()
modeLSTM_time.add(LSTM(units = 50,return_sequences=True, input_shape = (X_train.shape[1], 1)))

modeLSTM_time.add(TimeDistributed(Dense(10)))
modeLSTM_time.add(LSTM(units = 10))
modeLSTM_time.add(Dense(units=1))
modeLSTM_time.summary()

In [ ]:
modeLSTM_time.compile(optimizer = 'adam', loss = 'mean_squared_error')
modeLSTM_time.fit(X_train, y_train, epochs = 300, batch_size = 32);

In [ ]:
predict = modeLSTM_time.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test

print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))
print("MAPE:", np.mean(np.abs((test - predict) / test)) * 100)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[450:], df_volume[450:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df['Date'][-predict.shape[0]:].values, predict, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[450:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

--------------------

## Модель GRU з шаром TimeDistributed

In [ ]:
modeGRU_time = Sequential()
modeGRU_time.add(GRU(units = 50,return_sequences=True, input_shape = (X_train.shape[1], 1)))

modeGRU_time.add(TimeDistributed(Dense(10)))
modeGRU_time.add(GRU(units=10))
modeGRU_time.add(Dense(units=1))
modeGRU_time.summary()

In [ ]:
modeGRU_time.compile(optimizer = 'adam', loss = 'mean_squared_error')
modeGRU_time.fit(X_train, y_train, epochs = 300, batch_size = 16);

In [ ]:
predict = modeGRU_time.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test


print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))
print("MAPE:", np.mean(np.abs((test - predict) / test)) * 100)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[450:], df_volume[450:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df['Date'][-predict.shape[0]:].values, predict, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[450:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

-------------

## Модель LSTM з декількома рекурентними шарами

In [ ]:
modeLSTM_deep = Sequential()
modeLSTM_deep.add(LSTM(units = 50,return_sequences=True, input_shape = (X_train.shape[1], 1)))
modeLSTM_deep.add(Dense(units=50))
modeLSTM_deep.add(LSTM(units = 40, return_sequences = True))
modeLSTM_deep.add(Dense(units=40))
modeLSTM_deep.add(LSTM(units = 30, return_sequences = True))
modeLSTM_deep.add(TimeDistributed(Dense(1)))
modeLSTM_deep.add(LSTM(units = 1))
modeLSTM_deep.add(Dense(units=1))
modeLSTM_deep.summary()

In [ ]:
modeLSTM_deep.compile(optimizer = 'adam', loss = 'mean_squared_error')
modeLSTM_deep.fit(X_train, y_train, epochs = 150, batch_size = 32);

In [ ]:
predict = modeLSTM_deep.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test

print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))
print("MAPE:", np.mean(np.abs((test - predict) / test)) * 100)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[450:], df_volume[450:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df['Date'][-predict.shape[0]:].values, predict, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[450:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

-----------------------------

## Модель GRU з декількома рекрентними шарами

In [ ]:
modeGRU_deep = Sequential()
modeGRU_deep.add(GRU(units = 50,return_sequences=True, input_shape = (X_train.shape[1], 1)))
modeGRU_deep.add(Dense(units=50))
modeGRU_deep.add(GRU(units = 40, return_sequences = True))
modeGRU_deep.add(Dense(units=40))
modeGRU_deep.add(GRU(units = 30, return_sequences = True))
modeGRU_deep.add(TimeDistributed(Dense(1)))
modeGRU_deep.add(GRU(units = 1))
modeGRU_deep.add(Dense(units=1))
modeGRU_deep.summary()

In [ ]:
modeGRU_deep.compile(optimizer = 'adam', loss = 'mean_squared_error')
modeGRU_deep.fit(X_train, y_train, epochs = 150, batch_size = 16);

In [ ]:
predict = modeGRU_deep.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test

print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))
print("MAPE:", np.mean(np.abs((test - predict) / test)) * 100)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[450:], df_volume[450:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df['Date'][-predict.shape[0]:].values, predict, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[450:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

-------------------

## Модель LSTM з шаром Dropout

In [ ]:
modeLSTM_deepdrop = Sequential()
modeLSTM_deepdrop.add(LSTM(units = 50,return_sequences=True, input_shape = (X_train.shape[1], 1)))
modeLSTM_deepdrop.add(Dropout(0.2))

modeLSTM_deepdrop.add(LSTM(units = 50, return_sequences = True))
modeLSTM_deepdrop.add(Dropout(0.2))

modeLSTM_deepdrop.add(LSTM(units = 50, return_sequences = True))
modeLSTM_deepdrop.add(Dropout(0.2))

modeLSTM_deepdrop.add(TimeDistributed(Dense(1)))
modeLSTM_deepdrop.add(LSTM(units = 50))

modeLSTM_deepdrop.add(Flatten())
modeLSTM_deepdrop.add(Dense(50))
modeLSTM_deepdrop.add(Dropout(0.2))
modeLSTM_deepdrop.add(Dense(units=1))

modeLSTM_deepdrop.summary()

In [ ]:
modeLSTM_deepdrop.compile(optimizer = 'adam', loss = 'mean_squared_error')
modeLSTM_deepdrop.fit(X_train, y_train, epochs = 200, batch_size = 32);

In [ ]:
predict = modeLSTM_deepdrop.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test

print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))
print("MAPE:", np.mean(np.abs((test - predict) / test)) * 100)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[450:], df_volume[450:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df['Date'][-predict.shape[0]:].values, predict, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[450:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

> ### Передбачення на 20 днів

In [ ]:
pred_ = predict[-1].copy()
prediction_full = []
window = 60
df_copy = df.iloc[:, 1:2][1:].values

for j in range(20):
    df_ = np.vstack((df_copy, pred_))
    train_ = df_[:num_shape]
    test_ = df_[num_shape:]
    
    df_volume_ = np.vstack((train_, test_))

    inputs_ = df_volume_[df_volume_.shape[0] - test_.shape[0] - window:]
    inputs_ = inputs_.reshape(-1,1)
    inputs_ = sc.transform(inputs_)

    X_test_2 = []

    for k in range(window, num_2):
        X_test_3 = np.reshape(inputs_[k-window:k, 0], (window, 1))
        X_test_2.append(X_test_3)

    X_test_ = np.stack(X_test_2)
    predict_ = modelGRU.predict(X_test_)
    pred_ = sc.inverse_transform(predict_)
    prediction_full.append(pred_[-1][0])
    df_copy = df_[j:]

In [ ]:
prediction_full_new = np.vstack((predict, np.array(prediction_full).reshape(-1,1)))

df_date = df[['Date']]

for h in range(20):
    kk = pd.to_datetime(df_date['Date'].iloc[-1]) + pd.DateOffset(days=1)
    kk = pd.DataFrame([kk.strftime("%Y-%m-%d")], columns=['Date'])
    df_date = df_date.append(kk)
df_date = df_date.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[300:], df_volume[300:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df_date['Date'][-prediction_full_new.shape[0]:].values, prediction_full_new, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df_date[300:].shape[0],20))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

--------------------

## Модель GRU з шаром Dropout

In [ ]:
modeGRU_deepdrop = Sequential()
modeGRU_deepdrop.add(GRU(units = 50,return_sequences=True, input_shape = (X_train.shape[1], 1)))
modeGRU_deepdrop.add(Dropout(0.2))

modeGRU_deepdrop.add(GRU(units = 50, return_sequences = True, input_shape=(X_train.shape[1],1)))
modeGRU_deepdrop.add(Dropout(0.2))

modeGRU_deepdrop.add(GRU(units = 50, return_sequences = True,input_shape=(X_train.shape[1],1)))
modeGRU_deepdrop.add(Dropout(0.2))

modeGRU_deepdrop.add(TimeDistributed(Dense(1)))
modeGRU_deepdrop.add(GRU(units = 50))

modeGRU_deepdrop.add(Flatten())
modeGRU_deepdrop.add(Dense(50))
modeGRU_deepdrop.add(Dropout(0.2))
modeGRU_deepdrop.add(Dense(units=1))

modeGRU_deepdrop.summary()

In [ ]:
modeGRU_deepdrop.compile(optimizer = 'adam', loss = 'mean_squared_error')
modeGRU_deepdrop.fit(X_train, y_train, epochs = 200, batch_size = 16);

In [ ]:
predict = modeGRU_deepdrop.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test

print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))
print("MAPE:", np.mean(np.abs((test - predict) / test)) * 100)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[450:], df_volume[450:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df['Date'][-predict.shape[0]:].values, predict, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[450:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

> ### Передбачення на 20 днів

In [ ]:
pred_ = predict[-1].copy()
prediction_full = []
window = 60
df_copy = df.iloc[:, 1:2][1:].values

for j in range(20):
    df_ = np.vstack((df_copy, pred_))
    train_ = df_[:num_shape]
    test_ = df_[num_shape:]
    
    df_volume_ = np.vstack((train_, test_))

    inputs_ = df_volume_[df_volume_.shape[0] - test_.shape[0] - window:]
    inputs_ = inputs_.reshape(-1,1)
    inputs_ = sc.transform(inputs_)

    X_test_2 = []

    for k in range(window, num_2):
        X_test_3 = np.reshape(inputs_[k-window:k, 0], (window, 1))
        X_test_2.append(X_test_3)

    X_test_ = np.stack(X_test_2)
    predict_ = modelGRU.predict(X_test_)
    pred_ = sc.inverse_transform(predict_)
    prediction_full.append(pred_[-1][0])
    df_copy = df_[j:]

In [ ]:
prediction_full_new = np.vstack((predict, np.array(prediction_full).reshape(-1,1)))

df_date = df[['Date']]

for h in range(20):
    kk = pd.to_datetime(df_date['Date'].iloc[-1]) + pd.DateOffset(days=1)
    kk = pd.DataFrame([kk.strftime("%Y-%m-%d")], columns=['Date'])
    df_date = df_date.append(kk)
df_date = df_date.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[300:], df_volume[300:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df_date['Date'][-prediction_full_new.shape[0]:].values, prediction_full_new, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df_date[300:].shape[0],20))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

-------------------

## Модель GRU з шаром Conv1D

In [ ]:
modeGRU_conv = Sequential()
modeGRU_conv.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))

modeGRU_conv.add(GRU(50, activation='relu', return_sequences=True,))
modeGRU_conv.add(Flatten())

modeGRU_conv.add(Dense(32))
modeGRU_conv.add(Dropout(0.3))
modeGRU_conv.add(Dense(1))

modeGRU_conv.summary()

In [ ]:
modeGRU_conv.compile(optimizer = 'adam', loss = 'mean_squared_error')
modeGRU_conv.fit(X_train, y_train, epochs = 200, batch_size = 16);

In [ ]:
predict = modeGRU_conv.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test

print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))
print("MAPE:", np.mean(np.abs((test - predict) / test)) * 100)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[450:], df_volume[450:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df['Date'][-predict.shape[0]:].values, predict, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df[450:].shape[0],200))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

> ### Передбачення на 20 днів

In [ ]:
pred_ = predict[-1].copy()
prediction_full = []
window = 60
df_copy = df.iloc[:, 1:2][1:].values

for j in range(20):
    df_ = np.vstack((df_copy, pred_))
    train_ = df_[:num_shape]
    test_ = df_[num_shape:]
    
    df_volume_ = np.vstack((train_, test_))

    inputs_ = df_volume_[df_volume_.shape[0] - test_.shape[0] - window:]
    inputs_ = inputs_.reshape(-1,1)
    inputs_ = sc.transform(inputs_)

    X_test_2 = []

    for k in range(window, num_2):
        X_test_3 = np.reshape(inputs_[k-window:k, 0], (window, 1))
        X_test_2.append(X_test_3)

    X_test_ = np.stack(X_test_2)
    predict_ = modelGRU.predict(X_test_)
    pred_ = sc.inverse_transform(predict_)
    prediction_full.append(pred_[-1][0])
    df_copy = df_[j:]

In [ ]:
prediction_full_new = np.vstack((predict, np.array(prediction_full).reshape(-1,1)))

df_date = df[['Date']]

for h in range(20):
    kk = pd.to_datetime(df_date['Date'].iloc[-1]) + pd.DateOffset(days=1)
    kk = pd.DataFrame([kk.strftime("%Y-%m-%d")], columns=['Date'])
    df_date = df_date.append(kk)
df_date = df_date.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(df['Date'].values[300:], df_volume[300:], color = 'red', label = 'Real Tesla Stock Price')
plt.plot(df_date['Date'][-prediction_full_new.shape[0]:].values, prediction_full_new, color = 'blue', label = 'Predicted Tesla Stock Price')
plt.xticks(np.arange(100,df_date[300:].shape[0],20))
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

------------------

## ЧАКФ та АКФ за стовпчиком "Price"

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
plot_acf(df['Price'])
plot_pacf(df['Price'])